In [ ]:
## Libraries
!pip install networkx numpy scipy dwave-samplers dimod graphistry cupy-cuda12x tqdm --quiet

In [ ]:
import os
import sys
import math
import time
import json
import itertools
from itertools import islice
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from scipy.stats import mode
from tqdm import tqdm
import graphistry

import sympy as sp
import networkx as nx
import dimod
from dwave.samplers import SimulatedAnnealingSampler

In [ ]:
# Flow conservation function
def add_flow_penalty(bqm, pos_vars, neg_vars, delta, lambda_):
    bqm.offset += lambda_ * delta ** 2
    for var in pos_vars + neg_vars:
        bqm.add_linear(var, lambda_)
    for i in range(len(pos_vars)):
        for j in range(i + 1, len(pos_vars)):
            bqm.add_quadratic(pos_vars[i], pos_vars[j], 2 * lambda_)
    for i in range(len(neg_vars)):
        for j in range(i + 1, len(neg_vars)):
            bqm.add_quadratic(neg_vars[i], neg_vars[j], 2 * lambda_)
    for p in pos_vars:
        for n in neg_vars:
            if p == n:
                continue
            bqm.add_quadratic(p, n, -2 * lambda_)
    for p in pos_vars:
        bqm.add_linear(p, -2 * delta * lambda_)
    for n in neg_vars:
        bqm.add_linear(n, 2 * delta * lambda_)

## extract path from sampler result
def extract_path(sample, start, target, edge_vars):
    selected_edges = [(u, v) for (u, v), var in edge_vars.items() if sample.get(var, 0) == 1]
    if not selected_edges:
        print("no egde")
        return None
    path_graph = nx.DiGraph(selected_edges)
    try:
        path = nx.shortest_path(path_graph, start, target)  # Or use all_simple_paths if multiples
        return path
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        print("no path")
        return None

## compute weight for each edge
def compute_weight(row):
    return row['cost'] + row['detection_prob'] - row['exploitability'] - row['power_gain']

In [ ]:
## Load nodes data and extract exposed and crtical nodes per zone
nodes_df = pd.read_csv('Nodes_2000.csv')
zones = sorted(nodes_df['zone'].unique())

start_per_zone = {}
target_per_zone = {}
for zone in zones:
    starts = nodes_df[(nodes_df['zone'] == zone) & (nodes_df['is_exposed_to_internet'] == 1)]['node_id'].tolist()
    targets = nodes_df[(nodes_df['zone'] == zone) & (nodes_df['criticality'] == 5)]['node_id'].tolist()
    start_per_zone[zone] = sorted(starts)
    target_per_zone[zone] = sorted(targets)
    print("zone:",zone, ",start_node:",len(starts), ",target_node:",len(targets))

In [ ]:
## Load the edges
df = pd.read_csv('GraphEdgesForQUBO_2000.csv')
df["weight"] = df.apply(compute_weight, axis=1)

## Build the graph
G = nx.DiGraph()
for _, row in df.iterrows():
    G.add_edge(row['src'], row['dst'], cost=row['cost'], detection_prob=row['detection_prob'],
               exploitability=row['exploitability'], power_gain=row['power_gain'],
               weight=row["weight"], eid=row["edge_id"])

In [ ]:
## use thrid party api to show the graph
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", personal_key_id="4RIRZ03YNF", personal_key_secret="7S4CMOKE2T6448SP")

edge_rows = []

for u, v, data in G.edges(data=True):
    edge_rows.append({
        "src": u,
        "dst": v,
        "cost": data.get("cost"),
        "detection_prob": data.get("detection_prob"),
        "exploitability": data.get("exploitability"),
        "power_gain": data.get("power_gain")
    })

edges_df = pd.DataFrame(edge_rows)

graphistry.bind(
    source="src",
    destination="dst",
    edge_title="exploitability",
    edge_weight="power_gain"
).plot(edges_df)


In [ ]:
## extract and saved all the paths between each start and target pair per zone
single_path_pairs = []
for zone in zones:
    starts = start_per_zone.get(zone, [])
    targets = target_per_zone.get(zone, [])
    if not starts or not targets:
        continue
    print(zone)
    for s in starts:
        for t in targets:
            if s == t:
                print("no")
                continue
            try:
                paths = list(nx.all_simple_paths(G, s, t))
                print("zone:",zone, ",start_node:",s, ",target_node:",t, ",paths:",len(paths))
                single_path_pairs.append((zone, s, t, paths))

            except (nx.NodeNotFound, nx.NetworkXNoPath):
                pass


# Print the lists
# print("Start nodes per zone:", start_per_zone)
print(len(start_per_zone))
# print("Target nodes per zone:", target_per_zone)
print(len(target_per_zone))
# print("Single path pairs (zone, start, target, path):", single_path_pairs)
print(len(single_path_pairs))

In [ ]:
## extract and saved the optimal paths between each start and target pair per zone
single_path_pairs_optimal = []
for zone in zones:
    starts = start_per_zone.get(zone, [])
    targets = target_per_zone.get(zone, [])
    if not starts or not targets:
        continue
    print(zone)
    for s in starts:
        for t in targets:
            if s == t:
                continue
            try:
                optimal_path = nx.bellman_ford_path(G, s, t, weight='weight')
                optimal_objective = nx.path_weight(G, optimal_path, 'weight')

                print("Optimal path:", optimal_path)
                print("Objective value:", optimal_objective)
                single_path_pairs_optimal.append((zone, s, t, optimal_path))

            except nx.NetworkXNoPath:
                print("No path exists between the given nodes.")

            except nx.NetworkXUnbounded:
                print("A negative weight cycle is reachable from the source.")

In [ ]:
edge_vars = {(u, v): f"{u}-{v}" for u, v in G.edges()}

all_amplify_energies = []
all_amplify_paths = []
amplify_validation_results = []

all_sas_energies = []
all_sas_paths = []
sas_validation_results = []

In [ ]:
# SAS
print(len(single_path_pairs))
for idx, (zone, start_node, target_node, path) in enumerate(single_path_pairs):
    print(f"\nProcessing pair {idx + 1}: {start_node} to {target_node} in zone {zone}")

    # Get all nodes and edges (use full graph since paths can cross zones)
    nodes = list(G.nodes())
    edge_vars = {(u, v): f"{u}-{v}" for u, v in G.edges()}

    # Initialize BQM
    bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)

    # Add objective: minimize sum x_e * (cost_e + detection_e - exploitability_e - power_gain_e)
    for u, v, data in G.edges(data=True):
        var = edge_vars[(u, v)]
        coeff = data['cost'] + data['detection_prob'] - data['exploitability'] - data['power_gain']
        bqm.add_linear(var, coeff)

    # Penalty strength
    lambda_penalty = 5000.0  # Adjust as needed

    # Add flow constraints
    for v in nodes:
        out_edges = list(G.out_edges(v))
        in_edges = list(G.in_edges(v))
        pos_vars = [edge_vars.get((u, w)) for u, w in out_edges if (u, w) in edge_vars]
        neg_vars = [edge_vars.get((p, q)) for p, q in in_edges if (p, q) in edge_vars]
        if v == start_node:
            delta = 1.0
        elif v == target_node:
            delta = -1.0
        else:
            delta = 0.0
        add_flow_penalty(bqm, pos_vars, neg_vars, delta, lambda_penalty)

    # SAS
    print("SAS")
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(bqm, num_reads=550, num_sweeps=1500, seed=42)
    best_sample = sampleset.first.sample
    all_sas_energies.append(sampleset.first.energy)
    all_sas_paths.append(extract_path(best_sample, start_node, target_node, edge_vars))


In [ ]:
## compare sampler results with all ground truth paths
for idx, (zone, start_node, target_node, gt_path) in enumerate(single_path_pairs):
    extracted_path = all_sas_paths[idx]

    if extracted_path is None:
        match = False
        reason = "No path extracted from SAS solution"
    elif extracted_path in gt_path:
        match = True
        reason = "Exact match"
        print("match")
    else:
        match = False
        reason = f"Mismatch: Extracted {extracted_path} vs Ground Truth {gt_path}"

    sas_validation_results.append({
        'index': idx + 1,
        'zone': zone,
        'start': start_node,
        'target': target_node,
        'match': match,
        'reason': reason
    })

# Print or display results
validation_df = pd.DataFrame(sas_validation_results)
validation_df.to_csv("sas_comparison.csv")

# Summary
num_matches = sum(1 for res in sas_validation_results if res['match'])
print(f"\nSummary: {num_matches}/{len(single_path_pairs)} pairs match the ground truth single path.")

In [ ]:
## compare sampler result with ground truth optimal path
for idx, (zone, start_node, target_node, gt_path) in enumerate(single_path_pairs_optimal):
    extracted_path = all_sas_paths[idx]

    if extracted_path is None:
        match = False
        reason = "No path extracted from SAS solution"
        print(reason)
    elif extracted_path == gt_path:
        match = True
        reason = "Exact match"
        print("match")
    else:
        match = False
        reason = f"Mismatch: Extracted {extracted_path} vs Optimal Truth {gt_path}"
        print(reason)

    sqa_validation_results.append({
        'index': idx + 1,
        'zone': zone,
        'start': start_node,
        'target': target_node,
        'match': match,
        'reason': reason
    })

# Print or display results
validation_df = pd.DataFrame(sqa_validation_results)
validation_df.to_csv("sqa_comparison_optimal.csv")

# Summary
num_matches = sum(1 for res in sqa_validation_results if res['match'])
print(f"\nSummary: {num_matches}/{len(single_path_pairs)} pairs match the optimal truth single path.")